# Cvičenie 5: Inferencia v expertných systémoch

Minulý týždeň ste implementovali jednoduchý expertný systém, ktorý mal pevne definované pravidlá. Síce takýto systém by sme nenazvali "inteligentným", vaša implementácia poukázala na spôsob použitia expertných systémov. Na dnešnom cvičení sa pozrieme na to, ako funguje inferenčný mechanizmus pre odvodenie nových znalostí a pre nájdenie riešení a odpovedí.

## Štruktúra expertných systémov

Ako sme už hovorili na poslednom cvičení, expertný systém sa skladá z troch základných komponentov. **Báza údajov** obsahuje fakty súvisiace s riešením určitého problému. Sú to údaje, ktoré slúžia ako vstup pre inferenčný mechanizmus. V niektorých prípadoch expertný systém môže vyžadovať ďalšie fakty od používateľa pre nájdenie riešenia.

**Báza znalostí** obsahuje tzv. produkčné pravidlá, ktoré inferenčný mechanizmus aplikuje na dáta z bázy údajov. Tieto produkčné pravidlá majú tvar **IF predpoklad THEN dôsledok**. Síce ich syntax je podobná vetveniam, fungujú nezávisle od seba (teda nemajú ELSE vetvu). Ich vyhodnotenie je takisto rozlišné od matematickej implikácie.

Pre vytvorenie predpokladu (a dôsledku) sú definované tri základné operátory:
* AND - a zároveň; dôsledok platí iba ak všetky podmienky v predpoklade sú splnené
* OR - alebo; dôsledok platí ak aspoň jedna podmienka v predpoklade je splnená
* NOT - negácia; dôsledok platí ak predpoklad nie je splnený.

Dôsledok reprezentuje zmenu v báze údajov po aplikácii pravidla a zvyčajne definuje akciu **ADD** (pridaj) alebo **REMOVE** (vymaž).

## Aplikácia produkčného pravidla

Inferenčný mechanizmus pri každom kroku vykonávania zisťuje pre každé jedno produkčné pravidlo, či je splnený jeho predpoklad. Napríklad majme produkčné pravidlo:

**IF (?x) is older than 18 THEN ADD (?x) is an adult**.

Zápis *(?x) is older than 18* je takzvaný vzor, do ktorého dosadíme hodnoty tak, aby sme našli zodpovedajúci fakt v báze údajov. Napríklad, ak máme bázu údajov:

* **Peter is older than 18**,
* **Cathy is older than 18**,
* **Cathy is an adult**,
* **Maurice is younger than 18**

tak vzor *(?x) is older than 18* bude splnený dvomi spôsobmi: *Peter is older than 18* a *Cathy is older than 18*. Inferenčný systém tak zistí, že produkčné pravidlo *IF (?x) is older than 18 THEN ADD (?x) is an adult* je potenciálne aplikovateľné pravidlo.

Pravidlo aplikujeme iba v prípade, že jeho dôsledok by zmenil bázu údajov. V predošlom prípade máme dve potenciálne aplikovateľné inštancie pravidla, z ktorých sa aplikuje iba jedna:

* **IF Peter is older than 18 THEN ADD Peter is an adult** - túto inštanciu aplikujeme, keďže fakt v dôsledku sa ešte nenachádza v báze údajov
* **IF Cathy is older than 18 THEN ADD Cathy is an adult** - túto inštanciu nemusíme aplikovať, pretože nezmenila by bázu údajov.

## Dopredné a spätné zreťazenie

Inferenčný mechanizmus dokáže pravidlá aplikovať dvoma spôsobmi: dopredným (*forward chaining*) a spätným zreťazením (*backward chaining*). Dopredné zreťazenie objaví všetky možné dôsledky aplikovaním produkčných pravidiel na báze údajov, kým spätné zreťazenie je viac cieľené a má zistiť, či sa vieme dopracovať k istému dôsledku z bázy údajov pomocou množiny produkčných pravidiel.

Ak si predstavíte expertný systém, ktorý pracuje so zoznamom ingrediencií a dostupnou zásobou jedál, ktoré máte doma, pomocou dopredného zreťazenia vám vie doporučiť jednoduchý recept, ktorý môžete navariť a pomocou spätného zreťazenia vám vie povedať, či dokážete navariť určité jedlo. Teda, pri doprednom zreťazení hľadáte také fakty, ktoré spĺňajú podmienky v predpoklade ľubovoľného produkčného pravidla. Pri spätnom zreťazení hľadáte také produkčné pravidlo, ktorého dôsledkom je fakt, ktorý sa snažíte potvrdiť.

Samotný mechanizmus inferencie funguje postupnou aplikáciou jednotlivých produkčných pravidiel pričom sa dodržuje ich poradie. Inferencia končí vtedy, ak sa už nemení počet faktov v báze údajov.

## Úloha 1: Príbuzenské vzťahy

Majme expertný systém, ktorý určuje príbuzenské vzťahy medzi skupinou ľudí podľa nasledujúcich pravidiel:

1. IF (?x) is male AND (?x) is a parent of (?y) THEN ADD (?x) is the father of (?y)
2. IF (?x) is female AND (?x) is a parent of (?y) THEN ADD (?x) is the mother of (?y)
3. IF (?x) is a parent of (?y) AND (?x) is a parent of (?z) THEN ADD (?y) is a sibling of (?z)
4. IF (?x) is a parent of (?y) AND (?y) is male THEN ADD (?y) is a son of (?x)
5. IF (?x) is a parent of (?y) AND (?y) is female THEN ADD (?y) is a daughter of (?x)
6. IF (?x) is a sibling of (?y) AND (?x) is male THEN ADD (?x) is a brother of (?y)
7. IF (?x) is a sibling of (?y) AND (?x) is female THEN ADD (?x) is a sister of (?y)

Báza údajov nech obsahuje nasledujúce fakty:
1. Cersei is female
2. Jaime is male
3. Tyrion is male
4. Tywin is male
5. Tywin is a parent of Cersei
6. Tywin is a parent of Jaime
7. Tywin is a parent of Tyrion
8. Joffrey is male
9. Cersei is a parent of Joffrey
10. Tommen is male
11. Cersei is a parent of Tommen

### Úloha 1.1: Dopredné zreťazenie

Vašou prvou úlohou je odvodiť všetky príbuzenské vzťahy na základe báze údajov a množinou produkčných pravidiel. Nižšie uvedieme iba postup pri aplikácii prvých dvoch pravidiel, odkaz na celkové riešenie nájdete na konci cvičenia.

Prvé pravidlo znie **IF (?x) is male AND (?x) is a parent of (?y) THEN ADD (?x) is the father of (?y)**. Najprv teda hľadáme také fakty, ktoré vieme použiť v predpoklade pravidla.

Prvý člen predpokladu je vzor **(?x) is male**, ktorý spĺňa päť faktov: *Jaime is male*, *Tyrion is male*, *Tywin is male*, *Joffrey is male*, *Tommen is male*. Druhý vzor je **(?x) is a parent of (?y)**, tento vzor spĺňajú fakty *Tywin is a parent of Cersei*, *Tywin is a parent of Jaime*, *Tywin is a parent of Tyrion*, *Cersei is a parent of Joffrey*, *Cersei is a parent of Tommen*.

Kombináciou dvoch vzorov dostaneme tieto aplikovateľné inštancie (musíme dodržať premenné v predpoklade):
1. **IF Tywin is male AND Tywin is a parent of Cersei THEN ADD Tywin is the father of Cersei**
2. **IF Tywin is male AND Tywin is a parent of Jaime THEN ADD Tywin is the father of Jaime**
3. **IF Tywin is male AND Tywin is a parent of Tyrion THEN ADD Tywin is the father of Tyrion**

Keďže žiaden z dôsledkov sa nenachádza v báze údajov, aplikujeme všetky tri a tak bázu údajov rozšírime o fakty *Tywin is the father of Cersei*, *Tywin is the father of Jaime* a *Tywin is the father of Tyrion*.

Pred aplikáciou druhého pravidla musíme najprv zistiť, či neviem znova aplikovať pravidlo číslo 1 na základe nových faktov. V tomto prípade to neplatí, takže môžeme prejsť na druhé pravidlo, ktoré znie **IF (?x) is female AND (?x) is a parent of (?y) THEN ADD (?x) is the mother of (?y)**.

Prvý vzor **(?x) is female** spĺňa jedine fakt *Cersei is female*, druhý vzor **(?x) is a parent of (?y)** spĺňajú fakty *Tywin is a parent of Cersei*, *Tywin is a parent of Jaime*, *Tywin is a parent of Tyrion*, *Cersei is a parent of Joffrey*, *Cersei is a parent of Tommen*.

Po kombinácii vzorov získame aplikovateľné inštancie:
1. **IF Cersei is female AND Cersei is a parent of Joffrey THEN ADD Cersei is the mother of Joffrey**
2. **IF Cersei is female AND Cersei is a parent of Tommen THEN ADD Cersei is the mother of Tommen**

Aplikujeme obe inštancie a tak rozšírime bázu údajov o fakty *Cersei is the mother of Joffrey* a *Cersei is the mother of Tommen*.

**Pokračujte takto aplikáciou zvyšných pravidiel, pred aplikáciou si ale vždy skontrolujte, či neviete znova aplikovať predošlé pravidlá.**

### Úloha 1.2: Spätné zreťazenie

Pri ukážke spätného zreťazenia našim cieľom bude zistiť, či Joffrey je brat Tommena, teda či platí fakt *Joffrey is a brother of Tommen*.

Najprv musíme nájsť také produkčné pravidlo, ktorého dôsledkom je nami hľadaný fakt. Takéto pravidlo je pravidlo č. 6, teda **IF (?x) is a sibling of (?y) AND (?x) is male THEN ADD (?x) is a brother of (?y)**. Toto pravidlo má dva vzory v predpoklade, po dosadení hodnôt premenných získame hľadané fakty *Joffrey is a sibling of Tommen* a *Joffrey is male*. Z týchto faktov druhý sa už nachádza v báze údajov (fakt číslo 8), potrebujeme potvrdiť prvý fakt, t.j. *Joffrey is a sibling of Tommen*.

Keďže fakt sme nenašli v báze údajov, hľadáme pravidlo, ktorého dôsledkom by bol nami hľadaný fakt. Takéto pravidlo je **IF (?x) is a parent of (?y) AND (?x) is a parent of (?z) THEN ADD (?y) is a sibling of (?z)**. Ak dosadíme hodnoty premenných, dostaneme pravidlo **IF (?x) is a parent of Joffrey AND (?x) is a parent of Tommen THEN ADD Joffrey is a sibling of Tommen**. Teraz potrebujeme potvrdiť vzory z predpokladu.

Prvý vzor (*(?x) is a parent of Joffrey*) spĺňa iba jeden fakt, fakt č. 9: *Cersei is a parent of Joffrey*. Druhý vzor (*(?x) is a parent of Tommen*) takisto spĺňa iba jeden fakt, fakt č. 11: *Cersei is a parent of Tommen*. V oboch inštanciách máme rovnakú hodnotu premennej *(?x)*, takže môžeme aplikovať inštanciu pravidla 3:

**IF Cersei is a parent of Joffrey AND Cersei is a parent of Tommen THEN ADD Joffrey is a sibling of Tommen**

Aplikáciou pravidla pridáme do báze údajov fakt *Joffrey is a sibling of Tommen*. Vďaka tomuto faktu vieme aplikovať inštanciu pravidla 6:

**IF Joffrey is a sibling of Tommen AND Joffrey is male THEN ADD Joffrey is a brother of Tommen**

Dôsledkom tohto pravidla je nami hľadaný fakt *Joffrey is a brother of Tommen*, takže sme potvrdili pravdivosť hľadaného faktu a môžeme ukončiť proces inferencie.

Vašou úlohou je pomocou spätného zreťazenia zistiť pravdivosť nasledujúcich faktov:
* *Cersei is a sister of Jaime*
* *Tyrion is a son of Tywin*
* *Jaime is the father of Joffrey*

### 1.3: Definícia pravidiel

Definujte nové pravidlá, ktoré určujú ďalšie príbuzenské vzťahy: strýko, teta, starý otec a stará matka.

## Úloha 2: Bonusová úloha

Navrhnite expertný systém pre riešenie reálneho problému. Expertný systém nech má minimálne 10 pravidiel a jeho funkčnosť ukážte na príklade dopredným aj spätným zreťazením. Ak vaše riešenie zdokumentujete a pošlete mailom do 20. 3. 2020, môžete za to získať max. 3 body do zápočtu. V dokumentácii popíšte riešený problém, zoznam produkčných pravidiel, bázu údajov a dva príklady - jeden pre dopredné (stačí jeden príklad pre každé pravidlo) a jeden pre spätné zreťazenie.

**Poznámka**: Ukážkové riešenie úlohy 1 nájdete [tu](solutions/lab5_solution.pdf).